In [1]:
import csv
import pandas as pd

import requests
from bs4 import BeautifulSoup, SoupStrainer
import re
import pandas as pd
import json
bussiness_licences = "business-licences-hackathon.csv"
file = open(bussiness_licences, "r")
reader = pd.read_csv(file, sep=';')

/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
# 35050 Business name is peoples names in brackets which are likely
# not to have results in canadabusinessregistries
df_no_nan = reader.dropna(subset=['BusinessName'])
businesses_names = df_no_nan[~df_no_nan.BusinessName.str.contains('(', regex=False)].BusinessName.unique()

# 35050 Business name is peoples names in brackets which are likely
# not to have results in canadabusinessregistries 
# so I am getting their business trade name instead. 
business_trade_names = df_no_nan[df_no_nan['BusinessName'].str.contains('(', regex=False)]["BusinessTradeName"].unique()

In [14]:
len(businesses_names)

58507

In [15]:
len(business_trade_names)

21606

In [17]:
from collections import defaultdict

def get_business_info(businesses, current_business_list):
    for business in businesses:
        temp_dict = {}

        temp_dict['Company_Name'] = business['Company_Name']

        if 'BN' in business:
            temp_dict['BN'] = business['BN']
        else:
            temp_dict['BN'] = "None"

        temp_dict['Juri_ID'] = business['Juri_ID']
        temp_dict['Status_State'] = business['Status_State']

        if 'Status_Notes' in business:
            temp_dict['Status_Notes'] = business['Status_Notes']
        else: 
            temp_dict['Status_Notes'] = "None"

        if 'Reg_office_province' in business:
            temp_dict['Reg_office_province'] = business['Reg_office_province']
        else:
            temp_dict['Reg_office_province'] = "None"

        if 'Reg_office_city' in business:
            temp_dict['Reg_office_city'] = business['Reg_office_city']
        else:
            temp_dict['Reg_office_city'] = "None"

        if 'Display_Date' in business:
            temp_dict['Display_Date'] = business['Display_Date']
        else:
            temp_dict['Display_Date'] = "None"
    
    current_business_list[business['MRAS_ID']] = temp_dict

In [18]:
current_business_list = defaultdict(dict)
company_name = "Amazon"
API = "https://searchapi.mrasservice.ca/Search/api/v1/search?fq=keyword:%7B"+ company_name +"&location=BC&lang=en&queryaction=fieldquery&sortfield=score&sortorder=desc"
response = requests.get(API)
content_json = response.json()
get_business_info(content_json['docs'], current_business_list)
    
current_business_list

defaultdict(dict,
            {'BC_BC0477546': {'Company_Name': 'AMAZON ENTERPRISES INC.',
              'BN': '899612295',
              'Juri_ID': 'BC0477546',
              'Status_State': 'Inactive',
              'Status_Notes': 'Dissolved for Failure to File',
              'Reg_office_province': 'BC',
              'Reg_office_city': 'VICTORIA',
              'Display_Date': '1994-07-27'}})

In [ ]:
current_business_list = defaultdict(dict)

# Deal with multiword business names
for business in businesses_names:
    business_name_list = business.split()
    if len(business_name_list) > 1:
        company_name = "%20".join(business_name_list)
        company_name += "%7D"
    else: 
        company_name = business
    API = "https://searchapi.mrasservice.ca/Search/api/v1/search?fq=keyword:%7B"+ company_name +"&location=BC&lang=en&queryaction=fieldquery&sortfield=score&sortorder=desc"
    response = requests.get(API)
    content_json = response.json()
    get_business_info(content_json['docs'], current_business_list)
    
    
    